## Create new bigquery table with the individual's mortgage that they can afford

In [15]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("BigqueryExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")


In [16]:
df_individuals = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.individuals")    # project_id.datatset.tablename. Use your project id
df_individuals.printSchema()

root
 |-- ID: long (nullable = true)
 |-- age: long (nullable = true)
 |-- has_spouse: boolean (nullable = true)
 |-- spouse_ID: long (nullable = true)
 |-- gross_salary: long (nullable = true)
 |-- has_student_loan: boolean (nullable = true)
 |-- student_loan_amount: long (nullable = true)
 |-- has_general_loan: boolean (nullable = true)
 |-- general_loan_amount: long (nullable = true)
 |-- has_alimony: boolean (nullable = true)
 |-- alimony_amount: long (nullable = true)



## Table lookout

In [17]:
df_spouse = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.spouse")   
df_spouse.printSchema()

root
 |-- spouse_ID: long (nullable = true)
 |-- age: long (nullable = true)
 |-- gross_salary: long (nullable = true)
 |-- has_student_loan: boolean (nullable = true)
 |-- student_loan_amount: long (nullable = true)
 |-- has_general_loan: boolean (nullable = true)
 |-- general_loan_amount: long (nullable = true)
 |-- has_alimony: boolean (nullable = true)
 |-- alimony_amount: long (nullable = true)



In [18]:
df_toetsinkomen = spark.read \
  .format("bigquery") \
  .load(" degroup11.group11dataset.toetsinkomen")   
df_toetsinkomen.show(100)

+------------+----------+
|Toetsinkomen|Toetsrente|
+------------+----------+
|           0|      17.5|
|       19500|      17.5|
|       20000|      18.5|
|       20050|      19.5|
|       21000|      20.5|
|       21500|      21.0|
|       22000|      22.0|
|       22500|      23.5|
|       23000|      24.5|
|       23500|      25.0|
|       24000|      25.5|
|       25000|      26.5|
|       26000|      27.0|
|       28000|      27.5|
|       55000|      28.0|
|       58000|      28.5|
|       61000|      29.0|
|       63000|      29.5|
|       65000|      30.0|
|       68000|      30.5|
|       70000|      31.0|
|       75000|      31.5|
|       77000|      32.5|
|       79000|      33.0|
|       85000|      33.5|
|       96000|      34.0|
|      110000|      34.5|
+------------+----------+



In [19]:
from pyspark.sql.functions import col, when


df_comb = df_individuals.alias("ind").join(df_spouse.alias("sp"), col("ind.ID") ==  col("sp.spouse_ID"),"inner")

## Create house spendable income and select important columns from dataframe

In [20]:
df = df_comb
# Convert necessary columns to appropriate data types (e.g., from string to integer/boolean)
df = df.withColumn("ind.gross_salary", col("ind.gross_salary").cast("int"))
df = df.withColumn("ind.alimony_amount", col("ind.alimony_amount").cast("int"))
df = df.withColumn("ind.has_spouse", col("ind.has_spouse") == "True")
df = df.withColumn("ind.student_loan_amount", col("ind.student_loan_amount").cast("int"))
df = df.withColumn("ind.general_loan_amount", col("ind.general_loan_amount").cast("int"))

df = df.withColumn("sp.gross_salary", col("sp.gross_salary").cast("int"))
df = df.withColumn("sp.alimony_amount", col("sp.alimony_amount").cast("int"))
df = df.withColumn("sp.student_loan_amount", col("sp.student_loan_amount").cast("int"))
df = df.withColumn("sp.general_loan_amount", col("sp.general_loan_amount").cast("int"))


df = df.withColumn("house_spendable_income", 
                   when(col("ind.has_spouse") == False, 
                        col("ind.gross_salary") - 12 * col("ind.alimony_amount"))
                   .otherwise(when(col("ind.gross_salary")>col("sp.gross_salary"),col("ind.gross_salary") - 12 * col("ind.alimony_amount")+ 1/3 * (col("sp.gross_salary")-12 * col("sp.alimony_amount")))\
                             .otherwise(col("sp.gross_salary") - 12 * col("sp.alimony_amount")+ 1/3 * (col("ind.gross_salary")-12 * col("ind.alimony_amount")))))



df = df.withColumn("house_spendable_income", col("house_spendable_income").cast("int"))
#df_comb.select("ind.student_loan_amount", "sp.student_loan_amount").show()
df.select("ind.ID", "ind.gross_salary","ind.alimony_amount","ind.has_spouse","ind.student_loan_amount", "ind.general_loan_amount", "sp.spouse_ID", "sp.gross_salary","sp.alimony_amount", "sp.student_loan_amount", "sp.general_loan_amount", "house_spendable_income").show()



+---+------------+--------------+----------+-------------------+-------------------+---------+------------+--------------+-------------------+-------------------+----------------------+
| ID|gross_salary|alimony_amount|has_spouse|student_loan_amount|general_loan_amount|spouse_ID|gross_salary|alimony_amount|student_loan_amount|general_loan_amount|house_spendable_income|
+---+------------+--------------+----------+-------------------+-------------------+---------+------------+--------------+-------------------+-------------------+----------------------+
|106|       75526|             0|      true|                  0|               9288|      106|       59122|             0|              58463|               1144|                 95233|
|153|      134600|          1199|     false|                  0|               5314|      153|      101412|             0|              69057|                  0|                120212|
|211|      144798|             0|      true|              26010|      

## Toetsinkomen: get for each row the corresponding toetsinkomen rente and add as column to df

In [21]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

def calculate_toetsrente(toetsinkomen_value):
    # Hard-coded Toetsinkomen values
    toetsinkomen_values = [0, 19500, 20000, 20050, 21000, 21500, 22000, 22500, 23000, 23500,
                            24000, 25000, 26000, 28000, 55000, 58000, 61000, 63000, 65000, 68000,
                            70000, 75000, 77000, 79000, 85000, 96000, 110000]
    
    # Find the closest Toetsinkomen value
    closest_toetsinkomen = min(toetsinkomen_values, key=lambda x: abs(x - toetsinkomen_value))
    
    # Corresponding Toetsrente values
    toetsrente_values = {0: 17.5, 19500: 17.5, 20000: 18.5, 20050: 19.5, 21000: 20.5,
                         21500: 21.0, 22000: 22.0, 22500: 23.5, 23000: 24.5, 23500: 25.0,
                         24000: 25.5, 25000: 26.5, 26000: 27.0, 28000: 27.5, 55000: 28.0,
                         58000: 28.5, 61000: 29.0, 63000: 29.5, 65000: 30.0, 68000: 30.5,
                         70000: 31.0, 75000: 31.5, 77000: 32.5, 79000: 33.0, 85000: 33.5,
                         96000: 34.0, 110000: 34.5}
    
    # Get the Toetsrente value for the closest Toetsinkomen
    toetsrente_value = toetsrente_values[closest_toetsinkomen]
    
    return toetsrente_value


interestCalcUDF = udf(lambda x:calculate_toetsrente(x),StringType())


df_income_w_interest = df.withColumn("toetsrente", interestCalcUDF(col("house_spendable_income")))
df_income_w_interest.select("ID", "ind.age", "house_spendable_income", "toetsrente").show()

+---+---+----------------------+----------+
| ID|age|house_spendable_income|toetsrente|
+---+---+----------------------+----------+
|106| 18|                 95233|      34.0|
|153| 18|                120212|      34.5|
|211| 18|                197999|      34.5|
|354| 18|                166440|      34.5|
|363| 18|                186763|      34.5|
|469| 18|                 72111|      31.0|
|520| 18|                193312|      34.5|
|539| 18|                179166|      34.5|
|594| 18|                194983|      34.5|
|  9| 19|                168304|      34.5|
| 30| 19|                114168|      34.5|
|267| 19|                158944|      34.5|
|306| 19|                 97033|      34.0|
|374| 19|                140441|      34.5|
|562| 19|                 49877|      28.0|
|638| 19|                106209|      34.5|
|758| 19|                206934|      34.5|
|828| 19|                211856|      34.5|
|836| 19|                 47206|      28.0|
|887| 19|                102180|

## Reduction in monthly payments due to loans and student loans

In [22]:
df_deduction_loan = df_income_w_interest.withColumn("deduction_loan_payment",
                                                    (col("ind.student_loan_amount")*0.0075 + col("ind.general_loan_amount") * 0.02)+ (col("sp.student_loan_amount")*0.0075 + col("sp.general_loan_amount") * 0.02))





## Calculate the maximum monthly money to spend on mortgage

In [23]:
df_monthly_spendable = df_deduction_loan.withColumn("max_to_spend_month", (col("house_spendable_income")*(1/100)*col("Toetsrente"))/12 - col("deduction_loan_payment"))
df_monthly_spendable.show(1)

+---+---+----------+---------+------------+----------------+-------------------+----------------+-------------------+-----------+--------------+---------+---+------------+----------------+-------------------+----------------+-------------------+-----------+--------------+----------------+------------------+--------------+-----------------------+-----------------------+---------------+-----------------+----------------------+----------------------+----------------------+----------+----------------------+------------------+
| ID|age|has_spouse|spouse_ID|gross_salary|has_student_loan|student_loan_amount|has_general_loan|general_loan_amount|has_alimony|alimony_amount|spouse_ID|age|gross_salary|has_student_loan|student_loan_amount|has_general_loan|general_loan_amount|has_alimony|alimony_amount|ind.gross_salary|ind.alimony_amount|ind.has_spouse|ind.student_loan_amount|ind.general_loan_amount|sp.gross_salary|sp.alimony_amount|sp.student_loan_amount|sp.general_loan_amount|house_spendable_incom

In [24]:
df_monthly_spendable.select("ind.ID", "sp.spouse_ID", "max_to_spend_month").show()

+---+---------+------------------+
| ID|spouse_ID|max_to_spend_month|
+---+---------+------------------+
|106|      106|2051.1558333333332|
|153|      153|2831.8875000000007|
|211|      211| 5075.041249999999|
|354|      354| 4225.025000000001|
|363|      363|        4899.29375|
|469|      469|1584.9675000000002|
|520|      520| 5060.837500000001|
|539|      539|4334.7525000000005|
|594|      594|        5474.52125|
|  9|        9|         4394.8475|
| 30|       30|3100.0699999999997|
|267|      267|3998.1050000000005|
|306|      306|2684.2683333333334|
|374|      374|3508.2237500000006|
|562|      562| 992.1966666666668|
|638|      638|2569.7512499999993|
|758|      758| 5378.080000000001|
|828|      828|         5520.0975|
|836|      836|495.33333333333337|
|887|      887|2807.8550000000005|
+---+---------+------------------+
only showing top 20 rows



# Determine final allowed mortgage for individual


In [25]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


def mor_calc(interest, payment, months):
    return int(payment * (1-((1+interest)**-months))/interest)


interest = 0.0037
months = 12 * 30


morCalcUDF = udf(lambda x:mor_calc(interest, x, months),StringType())


df2 = df_monthly_spendable.withColumn("possible_mortgage_amount", morCalcUDF(col("max_to_spend_month")))
df_mortgage_to_write = df2.select("ID", "sp.spouse_ID", "max_to_spend_month", "possible_mortgage_amount")
df_mortgage_to_write = df_mortgage_to_write.withColumn("max_to_spend_month", col("max_to_spend_month").cast("int")) 
df_mortgage_to_write.show()

+---+---------+------------------+------------------------+
| ID|spouse_ID|max_to_spend_month|possible_mortgage_amount|
+---+---------+------------------+------------------------+
|106|      106|              2051|                  407681|
|153|      153|              2831|                  562857|
|211|      211|              5075|                 1008700|
|354|      354|              4225|                  839754|
|363|      363|              4899|                  973769|
|469|      469|              1584|                  315023|
|520|      520|              5060|                 1005877|
|539|      539|              4334|                  861563|
|594|      594|              5474|                 1088100|
|  9|        9|              4394|                  873507|
| 30|       30|              3100|                  616161|
|267|      267|              3998|                  794652|
|306|      306|              2684|                  533517|
|374|      374|              3508|      

# Initialize table mortgage_amount_batch 
### with columns: ID, spouse_ID, max_to_spend_month,  mortgage_amount_batch

In [26]:
from google.cloud import bigquery

# Initialize the BigQuery client
client = bigquery.Client(project="degroup11")  

# Define the table schema
schema = [
    bigquery.SchemaField("ID", "INTEGER"),
    bigquery.SchemaField("spouse_ID", "INTEGER"),
    bigquery.SchemaField("max_to_spend_month", "INTEGER"),
    bigquery.SchemaField("possible_mortgage_amount", "INTEGER"),

]

# Specify the BigQuery dataset and table
dataset_id = "group11dataset"  
table_id = "mortgage_amount_batch"   

# Create the BigQuery table
table_ref = client.dataset(dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# Create the table if it doesn't exist
client.create_table(table, exists_ok=True)

Table(TableReference(DatasetReference('degroup11', 'group11dataset'), 'mortgage_amount_batch'))

## write dataframe to bigquery

In [27]:
df_mortgage_to_write.write.format("bigquery").\
option('table', "degroup11.group11dataset.mortgage_amount_batch").\
option("temporaryGcsBucket", "temp_degroup11"). \
mode("overwrite").save()

In [28]:
spark.stop()